# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import time

# Import API key
from config import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
cities_data = pd.read_csv("../WeatherPy/cities.csv")
cities_df = pd.DataFrame(cities_data)
cities_df.head()

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,Lüderitz,0,NaN,1612824397,54,-26.6481,15.1594,71.15,20.67
1,1,Ushuaia,0,AR,1612824397,59,-54.8000,-68.3000,64.40,11.50
2,2,Bredasdorp,5,ZA,1612824397,88,-34.5322,20.0403,62.60,2.30
3,3,Mount Gambier,0,AU,1612824397,77,-37.8333,140.7667,57.20,13.80
4,4,Cabo San Lucas,1,MX,1612824239,56,22.8909,-109.9124,75.00,4.61


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configuring gmaps
gmaps.configure(api_key=g_key)

In [4]:
# creating the locations and weight from the cities_df
locations = cities_df[["Lat", "Lng"]].astype(float)
my_weights = cities_df["Humidity"].astype(float)

In [5]:
fig = gmaps.figure(center=(30,20), zoom_level=1.5)

heat_layer = gmaps.heatmap_layer(locations, weights=my_weights, dissipating=False, max_intensity=my_weights.max(),
                                point_radius=1)

fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [8]:
# Ideal conditions: Temp between 70-80, wind speed < 10, cloudiness = 0 
# store into a variable called hotel_df
hotel_df = pd.DataFrame()
hotel_df = cities_df.loc[(cities_df["Max Temp"] >= 70) & (cities_df["Max Temp"] <= 90) 
                                    & (cities_df["Wind Speed"] < 10) & (cities_df["Cloudiness"] == 0)] 

hotel_df


,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
61,61,Siyabuswa,0,ZA,1612824404,64,-25.1132,29.0445,70.21,7.18
74,74,Saint-Pierre,0,RE,1612824407,69,-21.3393,55.4781,77.00,8.05
85,85,Saint-Philippe,0,RE,1612824408,69,-21.3585,55.7679,77.00,8.05
125,125,Kavaratti,0,IN,1612824412,64,10.5669,72.6420,81.14,7.81
188,188,Port Hedland,0,AU,1612824420,83,-20.3167,118.5667,75.20,9.22
190,190,Rayong,0,TH,1612824420,94,12.8333,101.4333,79.00,3.02
199,199,Corowa,0,AU,1612824421,52,-36.0333,146.3833,71.01,4.61
235,235,Salalah,0,OM,1612824426,73,17.0151,54.0924,71.60,2.30
237,237,Coihueco,0,CL,1612824426,53,-36.6167,-71.8333,78.80,3.44
309,309,Comonfort,0,MX,1612824434,12,20.7167,-100.7667,77.00,5.01


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [9]:
# adding new column to hotel_df
hotel_df["Hotel Name"] = ""

/Users/EricShaffer/opt/anaconda3/envs/PythonData/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [10]:
# create the base url for the api search 
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {
    "radius": 5000,
    "type": "hotel",
    "keyword": "hotel",
    "key": g_key
}

In [11]:
# make successive api calls for each city and find the nearest hotel within 5000 meters
for index, row in hotel_df.iterrows():
    # get the latitude and longitude
    lat = row["Lat"]
    lng = row["Lng"]
    
    #assign the location of the city (lat and lng)
    city_coord = f"{lat}, {lng}"
    
    # add this to the api
    params["location"] = city_coord
    
    # make the api call
    response = requests.get(base_url, params=params).json()
    # getting into the results section of the api     
    results = response['results']
 
    try:
        hotel_df.loc[index, "Hotel Name"] = results[0]["name"]
        print(f'The nearest hotel to {row["City"]} is {results[0]["name"]}')
        
    except:
        print("Could not find a hotel within range")
    
    time.sleep(1)

/Users/EricShaffer/opt/anaconda3/envs/PythonData/lib/python3.6/site-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


The nearest hotel to Siyabuswa is Kings Palace Empire
The nearest hotel to Saint-Pierre is Le Battant Des Lames
The nearest hotel to Saint-Philippe is Les Embruns Du Baril
The nearest hotel to Kavaratti is yaad Hotal
The nearest hotel to Port Hedland is Hedland Hotel
Could not find a hotel within range
The nearest hotel to Corowa is Circa 1936
The nearest hotel to Salalah is Grand flora hotel salalah
Could not find a hotel within range
Could not find a hotel within range
The nearest hotel to Coyhaique is The Patagonian Lodge
The nearest hotel to Keetmanshoop is Hotel Schützenhaus
The nearest hotel to Port Augusta is Sip'n Save - Pastoral Hotel
The nearest hotel to Imbituba is Praia Hotel Imbituba
The nearest hotel to Ancud is Mística Natural


In [12]:
hotel_df

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
61,61,Siyabuswa,0,ZA,1612824404,64,-25.1132,29.0445,70.21,7.18,Kings Palace Empire
74,74,Saint-Pierre,0,RE,1612824407,69,-21.3393,55.4781,77.00,8.05,Le Battant Des Lames
85,85,Saint-Philippe,0,RE,1612824408,69,-21.3585,55.7679,77.00,8.05,Les Embruns Du Baril
125,125,Kavaratti,0,IN,1612824412,64,10.5669,72.6420,81.14,7.81,yaad Hotal
188,188,Port Hedland,0,AU,1612824420,83,-20.3167,118.5667,75.20,9.22,Hedland Hotel
190,190,Rayong,0,TH,1612824420,94,12.8333,101.4333,79.00,3.02,
199,199,Corowa,0,AU,1612824421,52,-36.0333,146.3833,71.01,4.61,Circa 1936
235,235,Salalah,0,OM,1612824426,73,17.0151,54.0924,71.60,2.30,Grand flora hotel salalah
237,237,Coihueco,0,CL,1612824426,53,-36.6167,-71.8333,78.80,3.44,
309,309,Comonfort,0,MX,1612824434,12,20.7167,-100.7667,77.00,5.01,


In [21]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [22]:
# Add marker layer ontop of heat map
fig.add_layer(gmaps.symbol_layer(locations, fill_color='green',
    stroke_color='green', scale=2, info_box_content=hotel_info))

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))